<a href="https://colab.research.google.com/github/yeshurunlevi/yesl/blob/main/trip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

columns:

link - ID?

name - name

original_location - list of all

claimed - restaurant business claimed on TripAdvisor

awards - awards, when get.

popularity_detailed - popularity detailed ranking

popularity_generic - popularity generic ranking (among all places to eat in the area)

top_tags - top tag names (key words?)

price_level - level of prices in current currency (data scraped from www.tripadvisor.ie - so currency is €)

price_range - range of prices in current currency (ranges displayed on TripAdvisor seem to be quite unreliable)

meals - types of meal

cuisines - types of cuisine

special_diets - types of special diets

features - restaurant features

vegetarian_friendly - is the restaurant vegetarian friendly?

vegan_options - does the restaurant offer vegan options?

gluten_free - does the restaurant gluten-free options?

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv(r'/content/drive/MyDrive/Trip Advisor EU restaurants /tripadvisor_european_restaurants.csv')

<ipython-input-2-c856f44723e1>:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'/content/drive/MyDrive/Trip Advisor EU restaurants /tripadvisor_european_restaurants.csv')


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1083397 entries, 0 to 1083396
Data columns (total 42 columns):
 #   Column                             Non-Null Count    Dtype  
---  ------                             --------------    -----  
 0   restaurant_link                    1083397 non-null  object 
 1   restaurant_name                    1083396 non-null  object 
 2   original_location                  1083397 non-null  object 
 3   country                            1083397 non-null  object 
 4   region                             1033074 non-null  object 
 5   province                           742765 non-null   object 
 6   city                               682712 non-null   object 
 7   address                            1083397 non-null  object 
 8   latitude                           1067607 non-null  float64
 9   longitude                          1067607 non-null  float64
 10  claimed                            1081555 non-null  object 
 11  awards                  

In [4]:
df2 = df[df['country'] == 'Greece']
df2 = df2[df2['total_reviews_count']>=5]

#df2 = df2.applymap(lambda x: x.replace(" ", "") if isinstance(x, str) else x)
df2 = df2.applymap(lambda x: x.lower() if isinstance(x, str) else x)

df2['claimed'] = df2['claimed'].map({'Claimed': 1, 'Unclaimed': 0})


def succ(x):
  if x >= 4.5:
    return 1
  else:
    return 0
df2['success (target)'] = [succ(x) for x in df2['avg_rating']]

<ipython-input-4-c41f0c708620>:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df2 = df2.applymap(lambda x: x.lower() if isinstance(x, str) else x)


In [5]:
df2['success (target)'].value_counts()

,count
success (target),
1,16034
0,9469


In [6]:
# Extract only the ranking number, handling NaNs
df2["popularity_detailed"], df2["restaurants"] = df2["popularity_detailed"].str.extract(r'#(\d)'), df2["popularity_detailed"].str.extract(r'#(\d+)')
df2["popularity_generic"], df2["places"] = df2["popularity_generic"].str.extract(r'#(\d)'), df2["popularity_generic"].str.extract(r'#(\d+)')


In [7]:
df2['to_new_data'] = df2['top_tags'] + ',' +df2['meals']+ ',' + df2['cuisines'] + ',' + df2['special_diets']+ ',' + df2['features']+ ','+ df2['keywords']

In [8]:
import pandas as pd
from collections import Counter


# Function to split awards and count them
def count_awards(awards_column):
    # Split each entry in the column by commas and flatten the result
    all_awards = [award.strip() for entry in awards_column if type(entry) == str for award in entry.split(',')]

    # Use Counter to count occurrences of each award
    award_count = Counter(all_awards)

    return award_count

# Apply the function
top_counts = count_awards(df2['to_new_data'])

print(top_counts)

Counter({'mediterranean': 2051, 'greek': 1790, 'vegetarian friendly': 1452, 'seafood': 1369, 'seating': 1217, 'serves alcohol': 1217, 'table service': 1211, 'dinner': 1194, 'lunch': 1163, 'outdoor seating': 1143, 'vegan options': 1133, 'reservations': 1113, 'free wifi': 1092, 'mid-range': 1017, 'highchairs available': 930, 'gluten free options': 917, 'accepts credit cards': 889, 'wheelchair accessible': 832, 'full bar': 789, 'european': 725, 'takeout': 623, 'after-hours': 601, 'brunch': 544, 'wine and beer': 453, 'drinks': 398, 'accepts visa': 397, 'accepts mastercard': 388, 'salad': 353, 'breakfast': 346, 'pizza': 297, 'parking available': 282, 'international': 238, 'healthy': 230, 'bar': 228, 'accepts american express': 222, 'street parking': 207, 'italian': 184, 'waterfront': 168, 'cheap eats': 161, 'free off-street parking': 157, 'greek salad': 157, 'television': 156, 'digital payments': 151, 'cash only': 144, 'moussaka': 143, 'cafe': 142, 'fresh fish': 119, 'bread': 119, 'grill': 

In [10]:
valid_terms = {k: v for k, v in top_counts.items() if v >= 100}# threshold

# Ensure 'to_new_data' is treated as a string before applying .lower()
for term in valid_terms.keys():
    df2[term] = df2['to_new_data'].astype(str).apply(lambda x: 1 if term in x.lower() else 0)



In [11]:
set(df2['price_level'].values)
df2['price_level'] = df2['price_level'].map({'€': 1, '€€-€€€': 2.5, '€€€€': 4})

In [12]:
import ast
from datetime import datetime
# Function to extract opening and closing times
def extract_hours(opening_hours, day):
    if pd.isna(opening_hours):
        return None, None  # Handle NaN values
    try:
        hours_dict = ast.literal_eval(opening_hours)  # Convert string to dictionary
        if day in hours_dict and hours_dict[day]:  # Ensure key exists and has values
            open_time, close_time = hours_dict[day][0].split('-')
            open_time, close_time = datetime.strptime(open_time, '%H:%M').time(), datetime.strptime(close_time, '%H:%M').time()
            return open_time, close_time
    except (ValueError, SyntaxError, KeyError, IndexError):
        return None, None  # Handle invalid or missing data
    return None, None

# List of days
days = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]

# Extract opening and closing times for each day
for day in days:
    df2[f"{day}_op"], df2[f"{day}_clo"] = zip(*df2["original_open_hours"].apply(lambda x: extract_hours(x, day)))

# Drop the original column if no longer needed
df2.drop(columns=["original_open_hours"], inplace=True)


In [13]:
df2['default_language'] = df2['default_language'].map({'English': 0, 'All languages': 1})

In [14]:
def extract_prices(price_range):
    if pd.isna(price_range):
        return np.nan, np.nan  # Keep NaN for missing values
    else:
      prices = price_range.replace("€", "").replace(",", "").split("-")  # Remove € and split
    return [int(prices[0]), int(prices[1])]  # Convert to integers

# Apply function and create new columns
df2["min_price"] =[extract_prices(cell)[0] for cell in df2["price_range"]]
df2["max_price"] =[extract_prices(cell)[1] for cell in df2["price_range"]]


In [15]:
df2.to_csv('/content/drive/MyDrive/Trip Advisor EU restaurants /first_try.csv', index=False)

In [16]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25503 entries, 952974 to 986735
Columns: 112 entries, restaurant_link to max_price
dtypes: float64(22), int64(52), object(38)
memory usage: 22.0+ MB


In [17]:
import pandas as pd
from collections import Counter


# Function to split awards and count them
def count_awards(awards_column):
    # Split each entry in the column by commas and flatten the result
    all_awards = [award.strip() for entry in awards_column if type(entry) == str for award in entry.split(',')]

    # Use Counter to count occurrences of each award
    award_count = Counter(all_awards)

    return award_count

# Apply the function
top_counts = count_awards(df2['to_new_data'])

print(top_counts)

Counter({'mediterranean': 2051, 'greek': 1790, 'vegetarian friendly': 1452, 'seafood': 1369, 'seating': 1217, 'serves alcohol': 1217, 'table service': 1211, 'dinner': 1194, 'lunch': 1163, 'outdoor seating': 1143, 'vegan options': 1133, 'reservations': 1113, 'free wifi': 1092, 'mid-range': 1017, 'highchairs available': 930, 'gluten free options': 917, 'accepts credit cards': 889, 'wheelchair accessible': 832, 'full bar': 789, 'european': 725, 'takeout': 623, 'after-hours': 601, 'brunch': 544, 'wine and beer': 453, 'drinks': 398, 'accepts visa': 397, 'accepts mastercard': 388, 'salad': 353, 'breakfast': 346, 'pizza': 297, 'parking available': 282, 'international': 238, 'healthy': 230, 'bar': 228, 'accepts american express': 222, 'street parking': 207, 'italian': 184, 'waterfront': 168, 'cheap eats': 161, 'free off-street parking': 157, 'greek salad': 157, 'television': 156, 'digital payments': 151, 'cash only': 144, 'moussaka': 143, 'cafe': 142, 'fresh fish': 119, 'bread': 119, 'grill': 

In [18]:
# Function to clean and split addresses
def clean_and_split_address(address):
    address = address.lower().replace("greece", "").strip()  # Remove "Greece"

    # Ensure we have at least a street and city/zip
    parts = address.split(",", 1) if "," in address else [address, ""]

    street_info = parts[0].strip()  # Street + number
    city_zip = parts[1].strip() if len(parts) > 1 else ""  # City + ZIP if available

    # Extract ZIP code and city
    zip_code = []
    city = []
    for char in city_zip:
        if char.isdigit():
            zip_code.append(char)
        else:
            city.append(char)

    city = "".join(city).strip()
    zip_code = "".join(zip_code)

    # Extract Apartment Number and Street Name
    apartment_number = []
    street_name = []
    for char in street_info:
        if char.isdigit():
            apartment_number.append(char)
        else:
            street_name.append(char)

    street_name = "".join(street_name).strip()
    apartment_number = "".join(apartment_number)

    return pd.Series([apartment_number, street_name, city, zip_code])

# Apply function to DataFrame
df2[['Apartment Number', 'Street Name', 'City_2', 'Zip Code']] = df2['address'].apply(clean_and_split_address)


# Display results
df2[['Apartment Number', 'Street Name', 'City_2', 'Zip Code']]


,Apartment Number,Street Name,City_2,Zip Code
952974,13,sekeri,tolon,21056
952975,14,atkis st,tolon,21056
952976,59,sekeri,tolon,21056
952977,8,aktis,tolon,21056
952978,28,sekeri,tolon,21056
...,...,...,...,...
986731,,fodele,crete,71500
986732,,kalamaki,"chania town, crete",73100
986733,5,th km chania-kissamos old road,"kalamaki, chania town, crete",73500
986734,,peo kissamou chanion kalamaki beach,"kalamaki, chania town, crete",73100


In [19]:
def extract_city(address):
    if isinstance(address, str):  # Check if the value is a string
        parts = address.split(",", 1)  # Split at the first comma
        return parts[0].strip()  # Return the part before the comma
    return ""  # Return an empty string if the address is not a string

# Fill NaN values in 'city' column with values from 'new city' column
df2['city'] = df2['city'].fillna(df2['City_2'].apply(extract_city))

In [20]:
#to drop
droping = ['restaurant_link', 'original_location','country', 'restaurant_link', 'vegetarian_friendly', 'vegan_options', 'gluten_free', 'top_tags', 'meals'
, 'cuisines', 'special_diets', 'features', 'keywords', 'price_range', 'address', 'City_2']
df2 = df2.drop(columns=droping, axis=1)